# Bibliotecas

In [ ]:
import os
import glob
import shutil
import psycopg2
import pandas as pd
from google.colab import drive

# Arquivos do projeto


In [ ]:
# Área de trabalho do projeto
drive.mount('/content/drive')
workspace_path = '/content/drive/MyDrive/Workspace'

Mounted at /content/drive


In [ ]:
# Caminho para os arquivos dos dados
data_path = f'{workspace_path}/fiap/pos-tech/data-analytics/fase-4/dados'
data_path += '/analise-de-negocios/dados'

In [ ]:
# Pasta temporária
temp_path = 'tmp'
os.makedirs(temp_path, exist_ok=True)

In [ ]:
# Copiando os arquivos do backup do banco de dados
for part_file in glob.glob(f'{data_path}/anp_dump.tar.gz.part-*'):
    shutil.copy(part_file, temp_path)

In [ ]:
# Remontando o arquivo de backup no formato .tar.gz
!cat tmp/anp_dump.tar.gz.part-* > tmp/anp_dump.tar.gz

In [ ]:
# Descompactando o arquivo de backup do banco de dados
!tar -xzvf tmp/anp_dump.tar.gz

tmp/anp_dump.backup


# PostgreSQL

In [ ]:
# Instalação
!apt-get install -y postgresql

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 49 not upgraded.
Need to get 18.4 MB of archives.
After this operation, 51.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd6

In [ ]:
# Iniciando o serviço
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [ ]:
# Configurando o usuário postgres
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"

ALTER ROLE


In [ ]:
# Criando o banco de dados
!sudo -u postgres psql -c "CREATE DATABASE anp;"

CREATE DATABASE


In [ ]:
# Restaurando os dados para o PostgreSQL
!PGPASSWORD=postgres pg_restore -U postgres -d anp -h localhost -v tmp/anp_dump.backup

pg_restore: connecting to database for restore
pg_restore: creating TABLE "public.preco_combustivel"
pg_restore: processing data for table "public.preco_combustivel"


# Explorando os dados

In [ ]:
# Conexão com o PostgreSQL
conn = psycopg2.connect(
    host='localhost',
    port='5432',
    database='anp',
    user='postgres',
    password='postgres'
)

In [ ]:
# Carregando os dados do PostgreSQL em um dataframe Pandas
df = pd.io.sql.read_sql_query("""
  SELECT  regiao_sigla      AS regiao
          , estado_sigla    AS estado
          , municipio
          , bandeira
          , produto
          , valor_de_venda  AS valor_venda
          , data_da_coleta  AS data_coleta
  FROM    public.preco_combustivel
""", conn)

<ipython-input-14-a8c3dfb0f4f5>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql_query("""


In [ ]:
# Ajustando a coluna data_da_coleta
df['data_coleta'] = pd.to_datetime(df['data_coleta'], format='%Y-%m-%d')

# Separando ano e mês
df['ano'] = df['data_coleta'].dt.year
df['mes'] = df['data_coleta'].dt.month

In [ ]:
# Informações do dataframe
display(df.head())
print('\nLinhas e colunas:', df.shape, '\n')
display(df.info())
print('\nRegistros nulos:')
display(df.isnull().sum())
print('\nEstatística básica:')
display(df.describe().round(2))

,regiao,estado,municipio,bandeira,produto,valor_venda,data_coleta,ano,mes
0,N,AC,RIO BRANCO,VIBRA ENERGIA,GASOLINA,6.99,2022-01-03,2022,1
1,N,AC,RIO BRANCO,VIBRA ENERGIA,ETANOL,5.99,2022-01-03,2022,1
2,N,AC,RIO BRANCO,VIBRA ENERGIA,DIESEL,6.09,2022-01-03,2022,1
3,N,AC,RIO BRANCO,VIBRA ENERGIA,GASOLINA ADITIVADA,7.05,2022-01-03,2022,1
4,N,AC,RIO BRANCO,VIBRA ENERGIA,DIESEL S10,6.12,2022-01-03,2022,1



Linhas e colunas: (4401370, 9) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401370 entries, 0 to 4401369
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   regiao       object        
 1   estado       object        
 2   municipio    object        
 3   bandeira     object        
 4   produto      object        
 5   valor_venda  float64       
 6   data_coleta  datetime64[ns]
 7   ano          int32         
 8   mes          int32         
dtypes: datetime64[ns](1), float64(1), int32(2), object(5)
memory usage: 268.6+ MB


None


Registros nulos:


,0
regiao,0
estado,0
municipio,0
bandeira,0
produto,0
valor_venda,0
data_coleta,0
ano,0
mes,0



Estatística básica:


,valor_venda,data_coleta,ano,mes
count,4401370.00,4401370,4401370.00,4401370.00
mean,4.48,2020-05-29 04:40:07.617628672,2019.93,6.27
min,1.76,2018-01-01 00:00:00,2018.00,1.00
25%,3.52,2019-02-18 00:00:00,2019.00,3.00
50%,4.19,2020-03-25 00:00:00,2020.00,6.00
75%,5.20,2021-10-20 00:00:00,2021.00,9.00
max,9.65,2022-12-30 00:00:00,2022.00,12.00
std,1.30,NaN,1.45,3.38


In [ ]:
print('Produtos comercializados:')
print(sorted(df.produto.unique()))

Produtos comercializados:
['DIESEL', 'DIESEL S10', 'ETANOL', 'GASOLINA', 'GASOLINA ADITIVADA', 'GNV']


In [ ]:
print('Período dos dados (anos):')
print(sorted(df.ano.unique()))

Período dos dados (anos):
[2018, 2019, 2020, 2021, 2022]


In [ ]:
print('Análise dos produtos por ano:')
df[['ano', 'produto', 'valor_venda']].groupby(['produto', 'ano']) \
                                     .agg(['mean', 'min', 'max']) \
                                     .round(2)

Análise dos produtos por ano:


valor_venda            
                               mean   min   max
produto            ano                         
DIESEL             2018        3.50  2.80  5.20
                   2019        3.60  2.87  4.99
                   2020        3.42  2.45  5.09
                   2021        4.69  3.10  6.99
                   2022        6.63  3.14  9.00
DIESEL S10         2018        3.59  2.84  5.30
                   2019        3.69  2.79  5.09
                   2020        3.51  2.46  5.38
                   2021        4.74  2.80  6.96
                   2022        6.73  3.59  9.65
ETANOL             2018        3.13  1.90  4.98
                   2019        3.17  2.10  5.47
                   2020        3.18  1.80  5.15
                   2021        4.67  2.05  7.90
                   2022        4.85  2.49  7.98
GASOLINA           2018        4.45  3.37  6.29
                   2019        4.42  3.39  6.29
                   2020        4.28  2.87  5.90
                   2021        5.89  3.10  8.00
                   2022        6.32  3.49  8.99
GASOLINA ADITIVADA 2020        4.59  3.69  6.00
                   2021        6.04  3.46  8.99
                   2022        6.47  4.09  9.28
GNV                2018        2.73  1.76  4.10
                   2019        3.22  2.00  4.56
                   2020        3.17  2.00  4.78
                   2021        3.89  2.38  6.70
                   2022        5.04  3.17  7.99

In [ ]:
print('Análise da quantidade de bandeiras:')
df[['ano', 'bandeira', 'produto']].groupby(['ano', 'produto']) \
                                  .bandeira.nunique()

Análise da quantidade de bandeiras:


ano   produto           
2018  DIESEL                64
      DIESEL S10            66
      ETANOL                66
      GASOLINA              66
      GNV                   24
2019  DIESEL                59
      DIESEL S10            61
      ETANOL                63
      GASOLINA              64
      GNV                   22
2020  DIESEL                52
      DIESEL S10            57
      ETANOL                58
      GASOLINA              58
      GASOLINA ADITIVADA    43
      GNV                   23
2021  DIESEL                57
      DIESEL S10            57
      ETANOL                58
      GASOLINA              58
      GASOLINA ADITIVADA    57
      GNV                   22
2022  DIESEL                51
      DIESEL S10            60
      ETANOL                60
      GASOLINA              60
      GASOLINA ADITIVADA    58
      GNV                   24
Name: bandeira, dtype: int64